<img align="center" src="figures/course.png" width="800">

#                                    16720 (B) Neural Networks for Recognition - Assignment 3

     Instructor: Kris Kitani                       TAs: Qichen(Lead), Paritosh, Rawal, Yan, Zen, Wen-Hsuan

## Q2 Implement a Fully Connected Network (65 points + 10 Extra Credit)

**Please include all the write up answers below to theory.ipynb**

In [3]:
# Do Not Modify
# Do Not Import ANY other packages
import numpy as np

# use for a "no activation" layer
def linear(x):
    return x

def linear_deriv(post_act):
    return np.ones_like(post_act)

def tanh(x):
    return np.tanh(x)

def tanh_deriv(post_act):
    return 1-post_act**2

def relu(x):
    return np.maximum(x, 0)

def relu_deriv(x):
    return (x > 0).astype(np.float)

### Q2.1 Network Initialization

#### Q2.1.1 (3 points WriteUp)
Why is it not a good idea to initialize a network with all zeros? If you imagine that every layer has weights and biases, what can a zero-initialized network output after training?

<font color="red">**Please include the write up answer to theory.ipynb**</font>

#### Q2.1.2 (3 points Autograder)
Implement `initialize_weights` below to initialize neural network weights with [Xavier initialization](http://proceedings.mlr.press/v9/glorot10a/glorot10a.pdf), where $Var[w] = \frac{2}{n_{in}+ n_{out}} $ where $n$ is the dimensionality of the vectors and you use a **uniform distribution** to sample random numbers (see eq 16 in [Xavier initialization](http://proceedings.mlr.press/v9/glorot10a/glorot10a.pdf)).

In [2]:
def initialize_weights(in_size: int, out_size: int, params: dict, name: str='' ):
    '''
    Initialize the weights W and b for a linear layer Y = XW + b
    
    [input]
    * in_size -- the feature dimension of the input
    * out_size -- the feature dimension of the output
    * params -- a dictionary containing parameters
    * name -- name of the layer
    
    HINTS:
    (1) b should be a 1D array, not a 2D array with a singleton dimension
    '''
    W, b = np.random.uniform(low= -(np.sqrt(6)/ np.sqrt(in_size + out_size)), high=(np.sqrt(6)/ np.sqrt(in_size + out_size)), size=(in_size, out_size)), np.zeros((out_size))

    # YOUR CODE HERE
    #raise NotImplementedError()

    params['W' + name] = W
    params['b' + name] = b

In [3]:
params = {}
initialize_weights(2,25,params,'layer1')
initialize_weights(25,4,params,'output')
assert(params['Wlayer1'].shape == (2,25))
assert(params['blayer1'].shape == (25,))


#### Q2.1.3 (2 points WriteUp)
Why do we initialize with random numbers? Why do we scale the initialization depending on layer size (see near Fig 6 in [Xavier initialization](http://proceedings.mlr.press/v9/glorot10a/glorot10a.pdf))?

<font color="red">**Please include the write up answer to theory.ipynb**</font>

### Q2.2 Forward Propagation

The appendix (in `theory.jpynb`) has the math for forward propagation, we will implement it here.

#### Q2.2.1 (12 points Autograder)
Implement `sigmoid`, along with `forward` propagation for a single layer with an activation function, namely
$y = \sigma(X W + b)$, returning the output and intermediate results for an $N \times D$ dimension input $X$, with examples along the rows, data dimensions along the columns.

In [4]:
def sigmoid(X: np.ndarray):
    '''
    A sigmoid activation function
    
    [input]
    * X -- input data [N x D]
    
    [output]
    * res -- output after the sigmoid function
    '''
    res = sig= 1/ (np.exp(-X)+1)

    # YOUR CODE HERE
    #raise NotImplementedError()
    
    return res


In [5]:
test = sigmoid(np.array([-100,100]))
assert test.min() < 1e-3
assert test.max() > 1 - 1e-3


In [6]:
def forward(X: np.ndarray, params: dict, name: str='',
            activation: callable=sigmoid):
    """
    Do a forward pass

    [input]
    * X -- input data [N x D]
    * params -- a dictionary containing parameters
    * name -- name of the layer
    * activation -- the activation function (default is sigmoid)
    
    [output]
    * post_act -- output after a linear layer and activation
    """
    pre_act, post_act = None, None
    # get the layer parameters
    W = params['W' + name]
    b = params['b' + name]
    pre_act = X @ W +np.tile(b, (X.shape[0], 1)) 
    post_act = activation(pre_act)
    # YOUR CODE HERE
    #raise NotImplementedError()

    # store the pre-activation and post-activation values
    # these will be important in backprop
    params['cache_' + name] = (X, pre_act, post_act)

    return post_act

In [7]:
params = {'Wlayer1': np.random.rand(10, 25), 'blayer1': np.random.rand(25,)}
X = np.random.rand(3, 10)
y = forward(X, params, 'layer1')
assert 'cache_layer1' in params


#### Q2.2.2 (5 points Autograder)
Implement the `softmax` function. Be sure the use the numerical stability trick you derived in Q1.1 softmax.

In [8]:
def softmax(X: np.ndarray):
    """
    A softmax function.
    
    [input]
    * X -- input data [N x D]
    
    [output]
    * res -- values after softmax
    """
    
    X = X.T
    
    max = np.max(X,axis=0) 
    ex = np.exp(X - max)
    sum = np.sum(ex,axis=0)
    res = np.array(ex/sum)
    
    res =res.T
   


    # YOUR CODE HERE
    #raise NotImplementedError()

    return res


In [9]:
# np.random.seed(0)
# a = np.random.rand(3,3)
# b = np.random.rand(1, 3)
# print(softmax1(a).shape)
# from scipy.special import softmax

# print(softmax(a))


#### Q2.2.3 (5 points Autograder)
Implement `compute_loss_and_acc` to compute the accuracy of a set of labels, along with the scalar loss across the data.  The loss function generally used for classification is the cross-entropy loss.

$$L_{\textbf{f}}(\textbf{D}) = - \sum_{(\textbf{x}, \textbf{y})\in \textbf{D}}\textbf{y}\cdot\log(\textbf{f}(\textbf{x}))$$
Here $\textbf{D}$ is the full training dataset of data samples $\textbf{x}$ ($N\times 1$ vectors, N = dimensionality of data) and labels $\textbf{y}$ ($C\times 1$ one-hot vectors, C = number of classes).

In [10]:
def compute_loss_and_acc(y: np.ndarray, probs: np.ndarray):
    """
    Compute total loss and accuracy
    
    [input]
    * y -- one hot labels [N x C]
    * probs -- class probabities [N x C]
    
    [output]
    * loss -- cross-entropy loss
    * acc -- accuracy
    """
    loss, acc = None, None
    #print(y.shape)
    #print(probs.shape)
    # YOUR CODE HERE
    #raise NotImplementedError()
    n = y.shape[0]
    loss = -np.sum(y * np.log(probs))
    correct = 0
    for i in range(n):
        p_idx = np.argmax(probs[i, :])
        if y[i, p_idx] == 1:
            correct +=1 
    
    acc = correct/n 
    return loss, acc

### Q2.3 Backwards Propagation

#### Q2.3.1 (10 points Autograder)
Compute back-propagation for a single layer, given the original weights, the appropriate intermediate results, and given gradient with respect to the loss. You should return the gradient with respect to $X$ so you can feed it into the next layer. As a size check, your gradients should be the same dimensions as the original objects.

In [11]:
def sigmoid_deriv(post_act: np.ndarray):
    """
    Derivative of sigmoid.
    
    we give this to you because you proved it
    it's a function of post_act
    """
    res = post_act*(1.0-post_act)
    return res


def backwards(delta: np.ndarray, params: dict, name: str='',
              activation_deriv: callable=sigmoid_deriv):
    """
    Do a backwards pass

    [input]
    * delta -- errors to backprop
    * params -- a dictionary containing parameters
    * name -- name of the layer
    * activation_deriv -- the derivative of the activation_func
    
    [output]
    * grad_X -- gradient w.r.t X
    """
    grad_X, grad_W, grad_b = None, None, None
    grad_Z_w, grad_Z_b = None, None # gradient of Z wrt W
    grad_Z = None
    grad_b_Z = None
    # everything you may need for this layer
    W = params['W' + name]
    b = params['b' + name]
    X, pre_act, post_act = params['cache_' + name]

    # do the derivative through activation first
    derv = activation_deriv(post_act) 
    # then compute the derivative W,b, and X
    grad_Z = derv*delta # N x C (y - a)
    grad_Z_w = X # N x D
    grad_Z_X = W # D x C
    grad_b_Z = np.ones((grad_Z.shape[0], )) # N, 
    
    # YOUR CODE HERE
    grad_W = grad_Z_w.T @ grad_Z # (D x N) @ (N x C) = D x C
    
    grad_X = grad_Z @ grad_Z_X.T # (N x C) @ ( C x D) = N x D
    
    #print(grad_X)
    # grad_Z x grad_b_Z(one : N x 1)
    #  1 x N  @ N x C  =  1 x C  
    grad_b = ( grad_b_Z.T @ grad_Z ).flatten()
    #print(grad_b.shape, b.shape)
    # store the gradients
    params['grad_W' + name] = grad_W
    params['grad_b' + name] = grad_b
    return grad_X
    
    #Reference : https://medium.com/@pdquant/all-the-backpropagation-derivatives-d5275f727f60

In [12]:
# we use random values to test your implementation 
# independent of previous questions
n, c1, c2 = 5, 40, 20 
delta = np.random.rand(n, c2)
name = 'layer1'
params = {
    'W'+name: np.random.rand(c1, c2),
    'b'+name: np.random.rand(c2),
    'cache_'+name: (np.random.rand(n, c1), 
                     np.random.rand(n, c2), 
                     np.random.rand(n, c2))
}
print()
grad = backwards(delta, params, name, tanh_deriv)

assert 'grad_W' + name in params
assert 'grad_b' + name in params

assert params['grad_W'+name].shape == params['W'+name].shape
assert params['grad_b'+name].shape == params['b'+name].shape


### Q2.4 Convolutional Layer [Extra Credit]
**Note: We would recommend finishing other questions before attempting questions under this section (q2.4.1 and q2.4.2)**

For now we have worked with linear layer in fully-connected networks. In practice, convolutional layers are commonly used to extract image feature. You will implement the forward and backawad propagation for convolutional layer in this subsection. 

#### Q2.4.1 [Extra Credit](5 points Autograder)
Similar to Q2.2.1, implement `conv_forward` for a single convolutional layer with zero paddings.

In [6]:
def conv_forward(X: np.ndarray, params: dict, name: str='',
            stride: int=1, pad: int=0):
    """
    Do a forward pass for a convolutional layer

    [input]
    * X -- input data [N x C x H x W]
    * params -- a dictionary containing parameters
    * name -- name of the layer
    * stride, pad -- convolution parameters
    
    [output]
    * res -- output after a convolutional layer
    """
    res = None
    # get the layer parameters
    w = params['W' + name] # Conv Filter weights [F x C x HH x WW]
    b = params['b' + name] # Biases [F]
    #pad the image
    img_pad = np.pad(X, ((0,0), (0,0), (pad,pad), (pad,pad)), 'constant', constant_values = (0,0))
    
    
    def crosscorelation(window, w, b):
        #print(window.shape)
        #print(w.shape)
        #print(b.shape)
        Y = np.sum(window * w) + b
        return Y
    
    N , C , H , W = X.shape
    #print(X.shape)
    F , C , HH , WW = w.shape
    #print(w.shape)
    #Output dimension of convolution:
    Hn = int((H + 2*pad - HH)/stride) + 1
    Wn = int((W +2*pad - WW)/stride)+1
    
    res = np.zeros((N, F , Hn, Wn))
    
    for i in range(N):
        #print(img_pad.shape[1])
        window = img_pad [i,:,:,:]
        for f in range(F):
            for j in range(Hn):
                for k in range(Wn):
                
                    hs = j*stride 
                    ws = k*stride
                    #print(hs)
                    fil = window[:, hs: hs+HH ,ws :ws+WW ]
                    #print(fil.shape)
                    res[i, f, j, k] = crosscorelation(fil, w[f,:,:,:], b[f])
    
    # YOUR CODE HERE
    #raise NotImplementedError()

    # store the input and convolution parameters
    # these will be important in backprop
    params['cache_' + name] = (X, stride, pad)

    return res

In [7]:
x_shape = np.array((2, 3, 4, 4))
w_shape = np.array((3, 3, 4, 4))
x = np.linspace(-0.1, 0.5, num=np.prod(x_shape), dtype=np.float64).reshape(*x_shape)
w = np.linspace(-0.2, 0.3, num=np.prod(w_shape), dtype=np.float64).reshape(*w_shape)
b = np.linspace(-0.1, 0.2, num=3, dtype=np.float64)

params = {'WConv_layer1': w, 'bConv_layer1': b}
y = conv_forward(np.array(x), params, 'Conv_layer1', stride=2, pad=1)
assert 'cache_Conv_layer1' in params


y_ref = np.array([[[[-0.08759809, -0.10987781],
                              [-0.18387192, -0.2109216 ]],
                             [[ 0.21027089,  0.21661097],
                              [ 0.22847626,  0.23004637]],
                             [[ 0.50813986,  0.54309974],
                              [ 0.64082444,  0.67101435]]],
                            [[[-0.98053589, -1.03143541],
                              [-1.19128892, -1.24695841]],
                             [[ 0.69108355,  0.66880383],
                              [ 0.59480972,  0.56776003]],
                             [[ 2.36270298,  2.36904306],
                              [ 2.38090835,  2.38247847]]]], 
            )
assert y.shape == y_ref.shape

max_diff = np.max(np.abs((y_ref - y)))
base = (np.abs(y_ref) + np.abs(y)).clip(np.finfo(float).eps).max()
print(max_diff/base) # the difference should be less than 1e-8


1.01418245052412e-09


#### Q2.4.2 [Extra Credit](5 points Autograder)
Implement `conv_backword` for a single convolutional layer with zero paddings.
Compute back-propagation for a single convolutional layer, given the original weights, the cached input, and given gradient with respect to the loss. Similar to Q2.3.1, you should return the gradient with respect to $X$ so you can feed it into the next layer. As a size check, your gradients should be the same dimensions as the original objects.

In [16]:
def conv_backward(delta: np.ndarray, params: dict, name: str=''):
    """
    Do a backwards pass for a convolutional layer

    [input]
    * delta -- errors to backprop
    * params -- a dictionary containing parameters
    * name -- name of the layer
    
    [output]
    * grad_X -- gradient w.r.t X
    """
    grad_X, grad_W, grad_b = None, None, None
    # everything you may need for this layer
    W = params['W' + name]
    b = params['b' + name]
    X, stride, pad = params['cache_' + name]

    
    
    
    
    N , C , H , W = X.shape
    #print(X.shape)
    F , C , HH , WW = w.shape
    N, F , nH , nW = delta.shape
    #print(w.shape)
    
    
    grad_X = np.zeros((N,C, H, W))
    grad_W = np.zeros((F,C, HH, WW))
    grad_b = np.zeros((F))
    
    img_pad = np.pad(X, ((0,0), (0,0), (pad,pad), (pad,pad)), 'constant', constant_values = (0,0))
    gradxpad = np.pad(grad_X, ((0,0), (0,0), (pad,pad), (pad,pad)), 'constant', constant_values = (0,0))
    
    inw = np.zeros((w.shape))
    for i in range(HH):
        for j in range(WW):
            inw[:,:,i,j] = w[:,:,HH-i-1,WW-j-1]
    
    #print(inw)

   
    for i in range(N):  
        xn = img_pad[i, :, :, :]
        gradxn = gradxpad[i, :, :, :]
        for f in range(F): 
            for h in range(nH):  
                
                vs = stride * h
                ve = stride * h + HH
                for j in range(nW):  
                    
                    hs = stride * j
                    he = stride * j + WW
                    sliced = xn[:, vs:ve, hs:he]

                    gradxn[:, vs:ve, hs:he] += inw[f, :, :, :] * delta[i, f, h, j]

                    grad_W[f, :, :, :] +=  sliced * delta[i, f, h, j]
                    grad_b[f] +=  delta[i, f, h, j]

        grad_X[i, :, :, :] =  gradxn[:, pad:-pad, pad:-pad]
    # compute the derivative W,b, and X
    # YOUR CODE HERE
    #raise NotImplementedError()
    #print(grad_X)
    # store the gradients
    params['grad_W' + name] = grad_W
    params['grad_b' + name] = grad_b
    return grad_X

In [17]:
x = np.random.rand(5, 4, 16, 16)
w = np.random.rand(8, 4, 7, 7)
b = np.random.rand(8,)
dout = np.random.rand(5, 8, 8, 8)

params = {'WConv_layer1': w, 'bConv_layer1': b}
y = conv_forward(x, params, 'Conv_layer1', stride=2, pad=3)
dx = conv_backward(dout, params, 'Conv_layer1')
assert x.shape == dx.shape
assert params['grad_WConv_layer1'].shape == params['WConv_layer1'].shape
assert params['grad_bConv_layer1'].shape == params['bConv_layer1'].shape


### Q2.5 Training Loop
You will tend to see gradient descent in three forms: "normal", "stochastic" and "batch". "Normal" gradient descent aggregates the updates for the entire dataset before changing the weights. Stochastic gradient descent applies updates after every single data example. Batch gradient descent is a compromise, where random subsets of the full dataset are evaluated before applying the gradient update. 

#### Q2.5.1 (10 points Autograder)
Write a training loop that generates random batches, iterates over them for many iterations, does forward and backward propagation, and applies a gradient update step. Specifically, implement `get_random_batches` and `train` functions below.

In [ ]:
def get_random_batches(x: np.ndarray, y: np.ndarray, batch_size: int) -> list:
    """
    Split x and y into random batches
    
    [input]
    * x -- training samples
    * y -- training lables
    * batch_size -- batch size
    
    [output]
    * batches -- a list of [(batch1_x,batch1_y)...]
    """
    # 
    # return 
    batches = []
    #print(x.shape[0])
    #idx = np.arange(x.shape[0])
    #print(idx)
    #idx = np.random.permutation(idx)
    #print(idx)
    #xr = x[idx, :]
    #yr = y[idx, :]
    #print(x ,"\n",y,"\n")
    #print(xr,"\n" ,yr,"\n")
    num_b = x.shape[0]//batch_size
    for i in range(num_b + 1):
        idx = np.arange(x.shape[0])
        idx = np.random.permutation(idx)
        idx = idx[:batch_size]
        #print(idx)
        batches.append((x[idx, :], y[idx, :]))
    
        
    #print(batches)
    # YOUR CODE HERE
    #raise NotImplementedError()
    return batches


In [ ]:
n, c1, c2 = 20, 100, 5
batch_size = 3
x = np.random.rand(n, c1)
y = np.random.rand(n, c2)
batches = get_random_batches(x, y, batch_size)
assert type(batches) == list
assert len(batches) >= 6


In [ ]:
def train(x: np.ndarray, y: np.ndarray, params: dict, batch_size: int = 5,
          max_iters: int = 500, learning_rate: float=1e-3):
    
    """
    Train the network with two sequential layers: 
    (1) one layer named "layer1" with sigmoid activation
    (2) one layer named "output" with softmax activation

    [input]
    * x -- training samples
    * y -- training lables
    * params -- a dictionary containing initial parameters
    * batch_size -- batch size
    * max_iters -- total number of iterations
    * learning_rate -- learning rate
    
    [output]
    * total_loss, avg_acc -- loss and accuracy for the last iteration
    """

    batches = get_random_batches(x, y, batch_size)
    n_batch = x.shape[0]//batch_size +1
    for itr in range(max_iters):
        
        total_loss = 0
        avg_acc = 0
        for xb, yb in batches:

            # forward
            # YOUR CODE HERE
            post_act1 =forward(xb, params, 'layer1',sigmoid)
            post_act2 = forward(post_act1,params,'output',softmax)
            #raise NotImplementedError()
            loss , accuracy = compute_loss_and_acc(yb, post_act2)
            # loss
            # be sure to add loss and accuracy to epoch totals
            total_loss += loss 
            avg_acc += accuracy
            
            # YOUR CODE HERE
            #raise NotImplementedError()
            delta1 =  post_act2 - yb
            #Finding delta 1
            
            delta2 = backwards(delta1,params,'output',linear_deriv)
            grad_X = backwards(delta2,params,'layer1',sigmoid_deriv)
            # backward
            # YOUR CODE HERE
            
            #raise NotImplementedError()
            
            params['W' + 'output'] -= (learning_rate)*params['grad_W' + 'output']
            params['b' + 'output'] -= (learning_rate)*params['grad_b' + 'output']
            params['W' + 'layer1'] -= (learning_rate)*params['grad_W' + 'layer1']
            params['b' + 'layer1'] -= (learning_rate)*params['grad_b' + 'layer1']
            # apply gradient
            # YOUR CODE HERE
            #raise NotImplementedError()
        avg_acc = avg_acc/n_batch 
        #print("itr: {:02d} \t loss: {:.2f} \t acc : {:.2f}".format(itr, total_loss, avg_acc))    
        
        if itr % 100 == 0:
            print("itr: {:02d} \t loss: {:.2f} \t acc : {:.2f}".format(
                itr, total_loss, avg_acc))
            #print(params['W' + 'layer1'])
    return total_loss, avg_acc


In [ ]:
# Successulf implementation of dependent functions are required to get full score for the `train` function

# create inputs
g0 = np.random.multivariate_normal([3.6,40],[[0.05,0],[0,10]],10)
g1 = np.random.multivariate_normal([3.9,10],[[0.01,0],[0,5]],10)
g2 = np.random.multivariate_normal([3.4,30],[[0.25,0],[0,5]],10)
g3 = np.random.multivariate_normal([2.0,10],[[0.5,0],[0,10]],10)
x = np.vstack([g0,g1,g2,g3])

# create labels
y_idx = np.array([0 for _ in range(10)] + [1 for _ in range(10)] + [2 for _ in range(10)] + [3 for _ in range(10)])

# turn labels to one_hot
y = np.zeros((y_idx.shape[0],y_idx.max()+1))
y[np.arange(y_idx.shape[0]),y_idx] = 1

# parameters in a dictionary
params = {}
# initialize a layer
initialize_weights(2,25,params,'layer1')
initialize_weights(25,4,params,'output')

# train the two-layer neural network
total_loss, avg_acc = train(x, y, params, batch_size=5, max_iters=500, learning_rate=1e-3)
print("itr: {:02d} \t loss: {:.2f} \t acc : {:.2f}".format(500, total_loss, avg_acc))

# with default settings, you should get loss < 35 and accuracy > 75%
assert total_loss < 35 and avg_acc > 0.75


### Q2.6 Numerical Gradient Checker

#### Q2.6.1 (15 points Autograder)
Implement the `centeral_differences_gradient` function. Instead of using the analytical gradients computed from the chain rule, add $\epsilon$ offset to each element in the weights, and compute the numerical gradient of the loss with central differences. Central differences is just $\frac{f(x+\epsilon) - f(x-\epsilon)}{2 \epsilon}$. Remember, this needs to be done for each scalar dimension in all of your weights independently. 

In [ ]:
def centeral_differences_gradient(params: dict, eps = 1e-6):
    """
    Compute the estimated gradients using central difference
    
    Hint:
    please feel free to reuse the functions above
    """
    import copy

    for k, v in params.items():
        #print(k)
        if '_' in k:
            
            continue
        
        
        #print("\n Original v : \n", params[k])
        if "b" in k:
            for i in range(v.shape[0]):
                params_copy = copy.deepcopy(params)
                #print("\n",i)

                params_copy[k][i] -= eps
                
                
                #print("\n", v[i]  )
                

                post_act1 =forward(x, params_copy, 'layer1')
                post_act2 = forward(post_act1, params_copy,'output',softmax)
                loss1 , accuracy1 = compute_loss_and_acc(y, post_act2)
                params_copy1[k][i] += eps
                #print("accuracy1", accuracy1)
                #print("loss1", loss1*1e6)
                # we have a real parameter!
                # for each value inside the parameter
                #   subtract epsilon
                #   run the forward function by 

                params_copy1 = copy.deepcopy(params)
                

                params_copy1[k][i] += eps
                
                #print("\n", params_copy2[k][i]  )
                #print("\n", v[i]  )
                post1 =forward(x, params_copy1, 'layer1')
                post2 = forward(post1, params_copy1,'output',softmax)
                loss2 , accuracy2 = compute_loss_and_acc(y, post2)
                params_copy1[k][i] -= eps
                #print("\n diff in first layer", post_act1 -post1 , "\n")

                #print("loss2: ", loss2*1e6)
                
                #print("\n", (loss1-loss2)/(2*eps), "\n")
                #print(params['grad_' + k][i], "\n")
                params['grad_' + k][i] = (loss2-loss1)/(2*eps)
               


        if "W" in k:
            for i in range(v.shape[0]):
                for j in range(v.shape[1]):
                    
                    params_copy = copy.deepcopy(params)
                    #print("\n",i)

                    params_copy[k][i,j] -= eps


                    #print("\n", v[i]  )


                    post_act1 =forward(x, params_copy, 'layer1')
                    post_act2 = forward(post_act1, params_copy,'output', softmax)
                    loss1 , accuracy1 = compute_loss_and_acc(y, post_act2)
                    params_copy[k][i,j] += eps
                    #print("accuracy1", accuracy1)
                    #print("loss1", loss1*1e6)
                    # we have a real parameter!
                    # for each value inside the parameter
                    #   subtract epsilon
                    #   run the forward function by 

                    params_copy1 = copy.deepcopy(params)


                    params_copy1[k][i,j] += eps

                    #print("\n", params_copy2[k][i]  )
                    #print("\n", v[i]  )
                    post1 =forward(x, params_copy1, 'layer1')
                    post2 = forward(post1, params_copy1,'output', softmax)
                    loss2 , accuracy2 = compute_loss_and_acc(y, post2)

                    #print("\n diff in first layer", post_act1 -post1 , "\n")

                    #print("loss2: ", loss2*1e6)

                    #print((loss2-loss1)/(2*eps))
                    params['grad_' + k][i,j] = (loss2-loss1)/(2*eps)
                    params_copy1[k][i,j] -= eps

    #print("\n", params[k])
        #   get the loss
        #   add epsilon
        #   run the forward function
        #   get the loss
        #   compute derivative with central diffs
    
        # YOUR CODE HERE
        #raise NotImplementedError()


In [ ]:
# Compute the analytical gradients
h1 = forward(x,params,'layer1')
probs = forward(h1,params,'output',softmax)
delta1 = probs
delta1[np.arange(probs.shape[0]),y_idx] -= 1

delta2 = backwards(delta1,params,'output',linear_deriv)
backwards(delta2,params,'layer1',sigmoid_deriv)

import copy
params_orig = copy.deepcopy(params)

# Compute the estimated gradient using central difference
centeral_differences_gradient(params)

total_error = 0
for k in params.keys():
    if 'grad_' in k:
        # relative error
        err = np.abs(params[k] - params_orig[k])/np.maximum(np.abs(params[k]),np.abs(params_orig[k]))
        err = err.sum()
        total_error += err
# should be less than 1e-4
assert 0. < total_error < 1e-4